# How Does Fuel Efficiency Vary Across Different Origins, and Does Engine Size Play a Role?

**Data**: `vega-datasets/data/cars.json` (406 rows, 9 columns)

Two approaches compared:
- **With Draco Guidance**: Draco's constraint solver recommends the structural design (mark type, channels, scales), Claude adds polish
- **Without Draco**: Claude designs the visualization independently

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
import json
import altair as alt
import pandas as pd
from tools.vislang import load_data, vl_to_altair

df = load_data("vega-datasets/data/cars.json")
log_dir = "../logs/2026-02-27_191514_cars-efficiency-origin-engine"

print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Origin values: {df['origin'].unique().tolist()}")
df.describe()

Shape: (406, 9)
Columns: ['name', 'miles_per_Gallon', 'cylinders', 'displacement', 'horsepower', 'weight_in_lbs', 'acceleration', 'year', 'origin']
Origin values: ['USA', 'Europe', 'Japan']


,miles_per_Gallon,cylinders,displacement,horsepower,weight_in_lbs,acceleration
count,398.000000,406.000000,406.000000,400.000000,406.000000,406.000000
mean,23.514573,5.475369,194.779557,105.082500,2979.413793,15.519704
std,7.815984,1.712160,104.922458,38.768779,847.004328,2.803359
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000
25%,17.500000,4.000000,105.000000,75.750000,2226.500000,13.700000
50%,23.000000,4.000000,151.000000,95.000000,2822.500000,15.500000
75%,29.000000,8.000000,302.000000,130.000000,3618.250000,17.175000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000


In [2]:
# Draco 2 Recommendations (structural advice for the visualization)
recommendations = {
    "mark_type": "point",
    "encodings": [
        {"channel": "x", "field": "miles_per_Gallon", "binning": 10},
        {"channel": "y", "field": "origin"},
        {"channel": "size", "field": "displacement", "aggregate": "mean"}
    ],
    "scales": [
        {"channel": "x", "type": "linear", "zero": "true"},
        {"channel": "y", "type": "ordinal"},
        {"channel": "size", "type": "linear", "zero": "true"}
    ],
    "coordinates": "cartesian",
    "task": "summary",
    "cost": [25]
}
print("Draco recommends:")
print(f"  Mark type: {recommendations['mark_type']}")
for enc in recommendations['encodings']:
    desc = f"  {enc['channel']}: {enc.get('field', '(count)')}"
    if enc.get('binning'): desc += f" (binned, {enc['binning']} bins)"
    if enc.get('aggregate'): desc += f" (aggregate: {enc['aggregate']})"
    print(desc)
for s in recommendations['scales']:
    desc = f"  Scale {s['channel']}: {s['type']}"
    if s.get('zero'): desc += f" (zero={s['zero']})"
    print(desc)

Draco recommends:
  Mark type: point
  x: miles_per_Gallon (binned, 10 bins)
  y: origin
  size: displacement (aggregate: mean)
  Scale x: linear (zero=true)
  Scale y: ordinal
  Scale size: linear (zero=true)


## With Draco Guidance

Draco recommended a **point** mark with MPG binned on x, origin on y (ordinal), and mean displacement encoded as size. Claude enhanced this with color coding, titles, tooltips, and mark styling.

In [3]:
with open(f"{log_dir}/with_draco.vl.json") as f:
    draco_spec = json.load(f)

chart_draco = vl_to_altair(draco_spec, df)
chart_draco

alt.Chart(...)

## Without Draco

Claude independently chose a **scatterplot** with displacement on x, MPG on y, and color by origin — a more traditional approach with interactive legend selection.

In [4]:
with open(f"{log_dir}/without_draco.vl.json") as f:
    baseline_spec = json.load(f)

chart_no_draco = vl_to_altair(baseline_spec, df)
chart_no_draco

alt.Chart(...)

## Comparison

| Aspect | With Draco Guidance | Without Draco |
|--------|-------------------|---------------|
| **Mark** | Point (dot plot) | Circle (scatterplot) |
| **X axis** | MPG (binned, 10 bins) | Displacement |
| **Y axis** | Origin (ordinal) | MPG |
| **Size** | Mean displacement | Fixed (60) |
| **Color** | Origin | Origin |
| **Interactivity** | Tooltips | Legend selection + tooltips |
| **Aggregation** | Yes (summary task) | No (individual points) |

In [5]:
# Side-by-side: build clean charts without top-level config for concatenation
draco_clean = {k: v for k, v in draco_spec.items() if k not in ("data", "$schema", "config")}
baseline_clean = {k: v for k, v in baseline_spec.items() if k not in ("data", "$schema", "config")}

left = alt.Chart.from_dict({"data": {"values": []}, **draco_clean}).properties(data=df, title="With Draco Guidance")
right = alt.Chart.from_dict({"data": {"values": []}, **baseline_clean}).properties(data=df, title="Without Draco")

alt.hconcat(left, right).resolve_scale(color='independent', size='independent')

alt.HConcatChart(...)